# Behavioral Lab 4

Welcome to our 4th lab! 

So far, we have discussed some background related to the COVID-19 pandemic and saw some datasets, practiced data manipulation with `numpy` and `pandas`, and learned some visualization techniques. 

### Today's lab

1. Merge Datasets that contain the same column(s).
2. Append Datasets.
3. Review for Midterm.

In [1]:
# Prepare the environment first.

from datascience import *
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt 

# 1. Merge Data

## 1.1. No missing "keys"

Data contained in pandas objects can be combined together in a number of ways:
`pandas.merge` connects rows in DataFrames based on one or more "keys". 

Merge operations combine datasets by linking rows using one or more keys. These operations are central to relational databases. The merge function in pandas is the main entry point for using these algorithms on your data.

Let's first see a simplest dataset:

In [2]:
df1 = pd.DataFrame({'date': ['M1','M2','M3','M1','M2','M3','M1','M2','M3','M1','M2','M3'], 
                    'county':['A','A','A','B','B','B','C','C','C','D','D','D'],
                    'data1': range(12)})

In [3]:
df1

,date,county,data1
0,M1,A,0
1,M2,A,1
2,M3,A,2
3,M1,B,3
4,M2,B,4
5,M3,B,5
6,M1,C,6
7,M2,C,7
8,M3,C,8
9,M1,D,9


In [4]:
df2 = pd.DataFrame({'date': ['M1','M2','M3','M1','M2','M3','M1','M2','M3','M1','M2','M3'], 
                    'county':['A','A','A','B','B','B','C','C','C','D','D','D'],
                    'data2': range(14,26)})

In [5]:
df2

,date,county,data2
0,M1,A,14
1,M2,A,15
2,M3,A,16
3,M1,B,17
4,M2,B,18
5,M3,B,19
6,M1,C,20
7,M2,C,21
8,M3,C,22
9,M1,D,23


Both "df1" and "df2" contain the same "date" and "county" columns, with different data values data1 and data2. It is very common that we have this kind of datasets under the same discipline with different focuses. For instance, our COVID-19 case-report dataset has county-date information, and the POI-visit dataset also has county-date information. How do we merge the two together? We use the `merge` function.

In [6]:
pd.merge(df1,df2)

,date,county,data1,data2
0,M1,A,0,14
1,M2,A,1,15
2,M3,A,2,16
3,M1,B,3,17
4,M2,B,4,18
5,M3,B,5,19
6,M1,C,6,20
7,M2,C,7,21
8,M3,C,8,22
9,M1,D,9,23


## 1.2. Outer and Inner Merging Method

It is rare in reality that we have the "perfect match", which means that all the keys in the first dataset are the same as all the keys in the second dataset. 

Let's see what if "df2" is missing county D:

In [7]:
df2_mD = pd.DataFrame({'date': ['M1','M2','M3','M1','M2','M3','M1','M2','M3'], 
                       'county':['A','A','A','B','B','B','C','C','C'],
                       'data2': range(14,23)})

In [8]:
pd.merge(df1,df2_mD)

,date,county,data1,data2
0,M1,A,0,14
1,M2,A,1,15
2,M3,A,2,16
3,M1,B,3,17
4,M2,B,4,18
5,M3,B,5,19
6,M1,C,6,20
7,M2,C,7,21
8,M3,C,8,22


Only counties showed in the 2nd dataset is merged. 

Note: By default merge does an `inner` join; the keys in the result are the intersection, or the common set found in both tables. Other possible options are `left`, `right`, and `outer`. The outer join takes the union of the keys, combining the effect of applying both left and right joins:

In [9]:
pd.merge(df1,df2_mD,on=['date', 'county'],how='outer') # Use all key combinations observed in both tables together

,date,county,data1,data2
0,M1,A,0,14.0
1,M2,A,1,15.0
2,M3,A,2,16.0
3,M1,B,3,17.0
4,M2,B,4,18.0
5,M3,B,5,19.0
6,M1,C,6,20.0
7,M2,C,7,21.0
8,M3,C,8,22.0
9,M1,D,9,NaN


In [10]:
pd.merge(df1,df2_mD,on=['date', 'county'],how='inner') # Use only the key combinations observed in both tables

,date,county,data1,data2
0,M1,A,0,14
1,M2,A,1,15
2,M3,A,2,16
3,M1,B,3,17
4,M2,B,4,18
5,M3,B,5,19
6,M1,C,6,20
7,M2,C,7,21
8,M3,C,8,22


In [11]:
df2_m3 = pd.DataFrame({'date': ['M1','M2','M1','M2','M1','M2','M1','M2'], 
                       'county':['A','A','B','B','C','C','D','D'],
                       'data3': range(8)})

# Let's use a new dataset that is missing date M3.

In [12]:
pd.merge(df1,df2_m3,on=['date', 'county'],how='left') # Use all key combinations found in the left table

,date,county,data1,data3
0,M1,A,0,0.0
1,M2,A,1,1.0
2,M3,A,2,NaN
3,M1,B,3,2.0
4,M2,B,4,3.0
5,M3,B,5,NaN
6,M1,C,6,4.0
7,M2,C,7,5.0
8,M3,C,8,NaN
9,M1,D,9,6.0


With two 'partial' datasets, let's compare the following two ways of merging:

In [13]:
pd.merge(df2_mD,df2_m3,on=['date', 'county'],how='left')

,date,county,data2,data3
0,M1,A,14,0.0
1,M2,A,15,1.0
2,M3,A,16,NaN
3,M1,B,17,2.0
4,M2,B,18,3.0
5,M3,B,19,NaN
6,M1,C,20,4.0
7,M2,C,21,5.0
8,M3,C,22,NaN


In [14]:
pd.merge(df2_mD,df2_m3,on=['date', 'county'],how='right')

,date,county,data2,data3
0,M1,A,14.0,0
1,M2,A,15.0,1
2,M1,B,17.0,2
3,M2,B,18.0,3
4,M1,C,20.0,4
5,M2,C,21.0,5
6,M1,D,NaN,6
7,M2,D,NaN,7


What did you notice? 

On the other hand, we can also try merging dataset that contains only one key column.

This is also very common in real life. For instance, we may observe each state's population, which is not changing over a certain period (due to the data collection effort), or we may observe each county's stay-at-home order date, which is not varying over time. 

Let's use the simpliest datasets df3 (date-specific information) and df4 (county-specific information) to practice merge again:

In [15]:
df3 = pd.DataFrame({'date': ['M1','M2','M3'],'data4': ['X','Y','Z']})
df4 = pd.DataFrame({'county':['A','B','C','D'],'data5':['a','b','c','d']})

In [16]:
pd.merge(pd.merge(df1,df3),df4) # And yes, we can do it in one line.

,date,county,data1,data4,data5
0,M1,A,0,X,a
1,M2,A,1,Y,a
2,M3,A,2,Z,a
3,M1,B,3,X,b
4,M2,B,4,Y,b
5,M3,B,5,Z,b
6,M1,C,6,X,c
7,M2,C,7,Y,c
8,M3,C,8,Z,c
9,M1,D,9,X,d


# 2. Append Data

Sometimes, we need to append data. The difference between appending to merging is that appending is adding more observations to the original dataset. Let's see some simple examples:

In [17]:
df1 = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df1

,A,B
0,1,2
1,3,4


In [18]:
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))
df2

,A,B
0,5,6
1,7,8


In [19]:
df1.append(df2)

,A,B
0,1,2
1,3,4
0,5,6
1,7,8


Now, we append df2 after df1, although the index is not ordered. We can use the `ignore_index` method:

In [20]:
df1.append(df2, ignore_index=True)

,A,B
0,1,2
1,3,4
2,5,6
3,7,8


What would happen if one dataset contains more columns than the other one? Can we still append them together?

In [21]:
df3 = pd.DataFrame([[1, 2, 3], [3, 4, 5]], columns=list('ABC'))
df3

,A,B,C
0,1,2,3
1,3,4,5


In [32]:
df3.append(df2, ignore_index=True)

,A,B,C
0,1,2,3.0
1,3,4,5.0
2,5,6,NaN
3,7,8,NaN


In [22]:
df2.append(df3, ignore_index=True)

,A,B,C
0,5,6,NaN
1,7,8,NaN
2,1,2,3.0
3,3,4,5.0


In [23]:
df4 = pd.DataFrame([[1, 2], [3, 4]], columns=list('BC'))
df4

,B,C
0,1,2
1,3,4


In [24]:
df4.append(df2, ignore_index=True)

,B,C,A
0,1,2.0,NaN
1,3,4.0,NaN
2,6,NaN,5.0
3,8,NaN,7.0


# 3. Practice with the COVID-19 datasets

In [25]:
# Import the datasets

nyt = pd.read_csv('YData_SDS177/nyt_cases_no_name.csv')
policy = pd.read_csv('YData_SDS177/county_closure.csv')
poi = pd.read_csv('YData_SDS177/poi_vpc.csv')
state = pd.read_csv('YData_SDS177/state_char.csv')
weather = pd.read_csv('YData_SDS177/weather.csv')

## 3.1. Merge New York Time case reports and dwelling time data `nyt` with `weather`. 

In [26]:
# Merge nyt to weather
pd.merge(nyt,weather,on=['date', 'geoid'],how='outer')

,geoid,cases,deaths,date,dwell_time,precip,rmax,rmin,srad,tmin,tmax,wind_speed
0,1001,0,0,01jan2020,929.95764,1.533333,33.282757,21.980459,125.570110,56.439655,69.550690,4.846695
1,1001,0,0,02jan2020,789.15399,3.728736,79.139084,64.109192,41.706898,54.343792,63.124481,14.712356
2,1001,0,0,03jan2020,777.49365,46.093105,90.395401,65.051727,36.435631,54.256897,66.335518,7.821563
3,1001,0,0,04jan2020,923.18738,0.477011,82.219543,40.403450,106.547130,44.100346,65.789307,12.387999
4,1001,0,0,05jan2020,972.03754,0.000000,61.641380,22.212645,139.329880,37.233448,65.739655,4.610146
...,...,...,...,...,...,...,...,...,...,...,...,...
456283,56045,0,0,22may2020,463.05606,0.493000,66.789497,14.456000,312.361240,45.674149,79.312553,10.115985
456284,56045,0,0,23may2020,335.37607,0.595000,94.435997,32.949749,243.686490,41.720001,67.687248,13.563663
456285,56045,0,0,24may2020,320.40112,0.079000,99.798500,46.666500,229.169490,35.396149,57.006050,13.923251
456286,56045,0,0,25may2020,402.71588,0.000000,99.928001,41.561749,295.531490,36.512600,61.895748,7.971323


Note that both datasets contain the same geoid and date, so we have 456,288 rows in the merged dataset as before.

Then we try to merge the above dataset with the policy dataset:

In [27]:
# Merge the above one to policy

nyt_weather_pol=pd.merge(pd.merge(nyt,weather,on=['date', 'geoid'],how='outer'),
                         policy,on=['date', 'geoid'],how='outer')
nyt_weather_pol

,geoid,cases,deaths,date,dwell_time,precip,rmax,rmin,srad,tmin,tmax,wind_speed,soe,sip,scs
0,1001,0,0,01jan2020,929.95764,1.533333,33.282757,21.980459,125.570110,56.439655,69.550690,4.846695,0,0,0
1,1001,0,0,02jan2020,789.15399,3.728736,79.139084,64.109192,41.706898,54.343792,63.124481,14.712356,0,0,0
2,1001,0,0,03jan2020,777.49365,46.093105,90.395401,65.051727,36.435631,54.256897,66.335518,7.821563,0,0,0
3,1001,0,0,04jan2020,923.18738,0.477011,82.219543,40.403450,106.547130,44.100346,65.789307,12.387999,0,0,0
4,1001,0,0,05jan2020,972.03754,0.000000,61.641380,22.212645,139.329880,37.233448,65.739655,4.610146,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456283,56045,0,0,22may2020,463.05606,0.493000,66.789497,14.456000,312.361240,45.674149,79.312553,10.115985,1,0,1
456284,56045,0,0,23may2020,335.37607,0.595000,94.435997,32.949749,243.686490,41.720001,67.687248,13.563663,1,0,1
456285,56045,0,0,24may2020,320.40112,0.079000,99.798500,46.666500,229.169490,35.396149,57.006050,13.923251,1,0,1
456286,56045,0,0,25may2020,402.71588,0.000000,99.928001,41.561749,295.531490,36.512600,61.895748,7.971323,1,0,1


## 3.2. Select columns from POI data and merge with the "inner" method.

Recall that we selected 5 columns of POI data in Lab 2, those 5 sectors "dvc4451", "dvc4461", "dvc4471" , "dvc4522", "dvc7121" are important to reflect people's responses under the pandemic. 

Also, the POI data may not contain 456288 rows as the above datasets, to get only merged information, what method should we call in `merge`?

In [29]:
# First select "date", "geoid", "dvc4451", "dvc4461", "dvc4471" , "dvc4522", "dvc7121" columns from POI data,
# then merge it to the above dataset

poi5 = poi.loc[:, ['date', 'geoid', 'dvc4451','dvc4461','dvc4471','dvc4522','dvc7121']]

In [30]:
fulldata = pd.merge(nyt_weather_pol,poi5,on=['date', 'geoid'],how='inner')
fulldata

,geoid,cases,deaths,date,dwell_time,precip,rmax,rmin,srad,tmin,tmax,wind_speed,soe,sip,scs,dvc4451,dvc4461,dvc4471,dvc4522,dvc7121
0,1001,0,0,27jan2020,738.02844,0.062069,100.000000,48.973564,92.362068,31.897587,59.762413,5.330079,0,0,0,0.049954,0.075952,0.072609,0.001114,0.024513
1,1001,0,0,28jan2020,728.81317,0.104598,98.162071,28.690805,159.349430,31.821035,66.418274,3.607381,0,0,0,0.050696,0.069638,0.069452,0.001857,0.022470
2,1001,0,0,29jan2020,726.98364,3.879310,100.000000,68.740227,53.597702,23.870001,49.024483,7.101630,0,0,0,0.052182,0.070566,0.067038,0.002228,0.015785
3,1001,0,0,30jan2020,712.97601,0.759770,100.000000,61.648277,159.598850,24.155518,51.345863,6.865080,0,0,0,0.047168,0.064995,0.076695,0.002786,0.025255
4,1001,0,0,31jan2020,688.28394,0.000000,100.000000,56.803448,67.574715,26.981724,58.597588,6.181143,0,0,0,0.067409,0.071309,0.102136,0.004643,0.020799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368923,56045,0,0,20may2020,426.45914,6.702750,100.000000,35.446251,283.253750,45.692150,80.762901,13.751007,1,0,1,0.053030,0.007576,0.147727,0.064394,0.060606
368924,56045,0,0,21may2020,494.07458,0.009250,69.317749,17.871000,316.473750,41.286652,71.197250,11.159516,1,0,1,0.030303,0.000000,0.121212,0.056818,0.041667
368925,56045,0,0,22may2020,463.05606,0.493000,66.789497,14.456000,312.361240,45.674149,79.312553,10.115985,1,0,1,0.034091,0.000000,0.136364,0.075758,0.056818
368926,56045,0,0,23may2020,335.37607,0.595000,94.435997,32.949749,243.686490,41.720001,67.687248,13.563663,1,0,1,0.053030,0.007576,0.132576,0.094697,0.045455


Note that "state" dataset contains the key column of "State FIPS Code" that allows us to merge with the above dataset. 

Recall that the geoid is combined with state FIPS code and county FIPS code.

Let's
1. create a new column named "stateid" in "fulldata"
2. we select the 3rd, 5th, 7th, 9th columns of "state", and name it "sel_state"
3. merge "fulldata" with "sel_state"

We take a closer look of the state dataset first, the 3rd, 5th, 7th, 9th columns contain some important information to help us charaterize a state:

In [31]:
state.head(5)

,State Abbreviation,State FIPS Code,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018,State of emergency,Stay at home/ shelter in place,Date closed K-12 schools,Closed other non-essential businesses,Mandate face mask use by all individuals in public spaces,Mandate face mask use by employees in public-facing businesses
0,AK,2,1.11,737438,665384.0,1907,6.8,10.9,32.8,4453,3/11/2020,3/28/2020,3/16/20,3/28/2020,4/24/2020,4/24/2020
1,AL,1,93.24,4887871,52420.0,3261,5.6,16.8,43.1,54352,3/13/2020,4/4/2020,3/20/20,3/28/2020,7/16/2020,5/11/2020
2,AR,5,56.67,3013825,53179.0,2717,4.5,17.2,43.5,32336,3/11/2020,0,3/17/20,4/4/2020,7/20/2020,5/11/2020
3,AZ,4,62.91,7171646,113990.0,10007,5.4,14.0,39.1,59282,3/11/2020,3/31/2020,3/16/20,3/30/2020,0,5/8/2020
4,CA,6,241.65,39557045,163695.0,151278,5.5,12.8,33.3,268818,3/4/2020,3/19/2020,3/23/20,3/19/2020,6/18/2020,5/5/2020


In [32]:
fulldata['stateid'] = np.floor(fulldata['geoid']/1000)
fulldata.head(10)

,geoid,cases,deaths,date,dwell_time,precip,rmax,rmin,srad,tmin,...,wind_speed,soe,sip,scs,dvc4451,dvc4461,dvc4471,dvc4522,dvc7121,stateid
0,1001,0,0,27jan2020,738.02844,0.062069,100.000000,48.973564,92.362068,31.897587,...,5.330079,0,0,0,0.049954,0.075952,0.072609,0.001114,0.024513,1.0
1,1001,0,0,28jan2020,728.81317,0.104598,98.162071,28.690805,159.349430,31.821035,...,3.607381,0,0,0,0.050696,0.069638,0.069452,0.001857,0.022470,1.0
2,1001,0,0,29jan2020,726.98364,3.879310,100.000000,68.740227,53.597702,23.870001,...,7.101630,0,0,0,0.052182,0.070566,0.067038,0.002228,0.015785,1.0
3,1001,0,0,30jan2020,712.97601,0.759770,100.000000,61.648277,159.598850,24.155518,...,6.865080,0,0,0,0.047168,0.064995,0.076695,0.002786,0.025255,1.0
4,1001,0,0,31jan2020,688.28394,0.000000,100.000000,56.803448,67.574715,26.981724,...,6.181143,0,0,0,0.067409,0.071309,0.102136,0.004643,0.020799,1.0
5,1001,0,0,01feb2020,838.75049,0.067816,100.000000,34.622990,80.305748,32.278275,...,7.721287,0,0,0,0.070381,0.048839,0.085422,0.004271,0.004643,1.0
6,1001,0,0,02feb2020,927.57678,0.000000,57.474712,25.347126,165.840230,46.988621,...,9.266573,0,0,0,0.076695,0.026370,0.065367,0.002971,0.010214,1.0
7,1001,0,0,03feb2020,718.74103,0.157471,76.637932,34.291954,167.062070,47.209999,...,8.744621,0,0,0,0.043792,0.063591,0.060738,0.002685,0.021141,1.0
8,1001,0,0,04feb2020,733.93420,2.289655,88.551727,48.824139,96.263222,51.989311,...,10.865853,0,0,0,0.041779,0.070134,0.059899,0.001174,0.017450,1.0
9,1001,0,0,05feb2020,752.15656,34.621838,73.751724,55.744827,106.777010,63.618965,...,14.133838,0,0,0,0.038758,0.061409,0.060570,0.000839,0.015101,1.0


We select the integer part of the value, geoid/1000, to construct our stateid. We put the column name in `[column_name]` after the dataset name as `dataframe['column_name']` to add a new column to the dataframe.

In [33]:
sel_state = state.iloc[:,[1,2,4,6,8]]
sel_state=sel_state.rename(columns={'State FIPS Code':'stateid'})
sel_state.head(5)

,stateid,Population density per square miles,Square Miles,Percent Unemployed (2018),Percent at risk for serious illness due to COVID
0,2,1.11,665384.0,6.8,32.8
1,1,93.24,52420.0,5.6,43.1
2,5,56.67,53179.0,4.5,43.5
3,4,62.91,113990.0,5.4,39.1
4,6,241.65,163695.0,5.5,33.3


Since we know the selected column numbers: 2, 3, 5, 7, 9 (which translates to 1, 2, 4, 6, 8 columns of the dataframe as it always starts with 0), we can use `iloc[rows,columns]` function to do our selection. 

We use `:` for rows in `iloc[rows,columns]` above to tell Python that we want to select all the rows, we specify `[1,2,4,6,8]` in brackets `[]` to tell Python our selected column numbers. 

With two cleaned datasets, we merge them with the key column "stateid":

In [34]:
pd.merge(fulldata,sel_state,on=['stateid'])

,geoid,cases,deaths,date,dwell_time,precip,rmax,rmin,srad,tmin,...,dvc4451,dvc4461,dvc4471,dvc4522,dvc7121,stateid,Population density per square miles,Square Miles,Percent Unemployed (2018),Percent at risk for serious illness due to COVID
0,1001,0,0,27jan2020,738.02844,0.062069,100.000000,48.973564,92.362068,31.897587,...,0.049954,0.075952,0.072609,0.001114,0.024513,1.0,93.24,52420.0,5.6,43.1
1,1001,0,0,28jan2020,728.81317,0.104598,98.162071,28.690805,159.349430,31.821035,...,0.050696,0.069638,0.069452,0.001857,0.022470,1.0,93.24,52420.0,5.6,43.1
2,1001,0,0,29jan2020,726.98364,3.879310,100.000000,68.740227,53.597702,23.870001,...,0.052182,0.070566,0.067038,0.002228,0.015785,1.0,93.24,52420.0,5.6,43.1
3,1001,0,0,30jan2020,712.97601,0.759770,100.000000,61.648277,159.598850,24.155518,...,0.047168,0.064995,0.076695,0.002786,0.025255,1.0,93.24,52420.0,5.6,43.1
4,1001,0,0,31jan2020,688.28394,0.000000,100.000000,56.803448,67.574715,26.981724,...,0.067409,0.071309,0.102136,0.004643,0.020799,1.0,93.24,52420.0,5.6,43.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368923,56045,0,0,20may2020,426.45914,6.702750,100.000000,35.446251,283.253750,45.692150,...,0.053030,0.007576,0.147727,0.064394,0.060606,56.0,5.91,97813.0,3.9,36.4
368924,56045,0,0,21may2020,494.07458,0.009250,69.317749,17.871000,316.473750,41.286652,...,0.030303,0.000000,0.121212,0.056818,0.041667,56.0,5.91,97813.0,3.9,36.4
368925,56045,0,0,22may2020,463.05606,0.493000,66.789497,14.456000,312.361240,45.674149,...,0.034091,0.000000,0.136364,0.075758,0.056818,56.0,5.91,97813.0,3.9,36.4
368926,56045,0,0,23may2020,335.37607,0.595000,94.435997,32.949749,243.686490,41.720001,...,0.053030,0.007576,0.132576,0.094697,0.045455,56.0,5.91,97813.0,3.9,36.4


# 4. Review for Midterm

Check the Course Slides: Lec 6 for key functions.